
Step-1: Install PySpark

In [3]:
# Install PySpark
!pip install pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, hour, dayofweek, count, avg

Step-2: Download the Dataset

In [13]:
# Download the Excel dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx -O Online_Retail.xlsx

--2025-06-10 06:46:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘Online_Retail.xlsx’

Online_Retail.xlsx      [  <=>               ]  22.62M  72.6MB/s    in 0.3s    

2025-06-10 06:46:11 (72.6 MB/s) - ‘Online_Retail.xlsx’ saved [23715344]



Step-3: Install openpyxl to read Excel file

In [14]:
!pip install openpyxl pandas

Step-4: Convert Excel to CSV using Pandas

In [15]:
import pandas as pd

# Read Excel and save as CSV
df_excel = pd.read_excel("Online_Retail.xlsx")
df_excel.to_csv("Online_Retail.csv", index=False)

Step-5: Start Spark Session and Load CSV

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, desc

# Initialize Spark session
spark = SparkSession.builder.appName("Retail Analysis").getOrCreate()

# Load the CSV file
df = spark.read.csv("Online_Retail.csv", header=True, inferSchema=True)

# Show schema and a few records
df.printSchema()
df.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365| 

Step-6: Basic Cleaning

In [17]:
# Remove rows with nulls in critical fields
df_clean = df.dropna(subset=["InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country"])

# Filter out cancellations (invoices with 'C' usually indicate canceled orders)
df_clean = df_clean.filter(~df_clean["InvoiceNo"].startswith("C"))

Step-7: Add Total Price Column

In [18]:
# Add a new column 'TotalPrice' = Quantity * UnitPrice
df_clean = df_clean.withColumn("TotalPrice", col("Quantity") * col("UnitPrice"))

Step-8: Top 10 Countries by Revenue

In [19]:
# Group by country and sum total price
country_sales = df_clean.groupBy("Country").agg(sum("TotalPrice").alias("TotalRevenue"))

# Show top 10 countries by revenue
country_sales.orderBy(desc("TotalRevenue")).show(10)

+--------------+------------------+
|       Country|      TotalRevenue|
+--------------+------------------+
|United Kingdom| 7308391.554000224|
|   Netherlands| 285446.3399999992|
|          EIRE|265545.89999999915|
|       Germany|228867.14000000025|
|        France|209024.05000000025|
|     Australia|138521.30999999976|
|         Spain| 61577.11000000017|
|   Switzerland| 56443.95000000007|
|       Belgium| 41196.34000000001|
|        Sweden| 38378.32999999999|
+--------------+------------------+
only showing top 10 rows



Step-9: Top 10 Customers by Spend

In [21]:
top_customers = df_clean.groupBy("CustomerID").agg(sum("TotalPrice").alias("CustomerSpend"))
top_customers.orderBy(desc("CustomerSpend")).show(10)

+----------+------------------+
|CustomerID|     CustomerSpend|
+----------+------------------+
|   14646.0| 280206.0199999994|
|   18102.0|259657.29999999996|
|   17450.0|194550.78999999995|
|   16446.0|          168472.5|
|   14911.0|143825.05999999988|
|   12415.0| 124914.5299999998|
|   14156.0|117379.62999999992|
|   17511.0| 91062.37999999989|
|   16029.0| 81024.84000000001|
|   12346.0|           77183.6|
+----------+------------------+
only showing top 10 rows



Step-10: Most Popular Products(by Quantity Sold)

In [22]:
popular_products = df_clean.groupBy("Description").agg(sum("Quantity").alias("TotalQuantity"))
popular_products.orderBy(desc("TotalQuantity")).show(10, truncate=False)

+----------------------------------+-------------+
|Description                       |TotalQuantity|
+----------------------------------+-------------+
|PAPER CRAFT , LITTLE BIRDIE       |80995        |
|MEDIUM CERAMIC TOP STORAGE JAR    |77916        |
|WORLD WAR 2 GLIDERS ASSTD DESIGNS |54415        |
|JUMBO BAG RED RETROSPOT           |46181        |
|WHITE HANGING HEART T-LIGHT HOLDER|36725        |
|ASSORTED COLOUR BIRD ORNAMENT     |35362        |
|PACK OF 72 RETROSPOT CAKE CASES   |33693        |
|POPCORN HOLDER                    |30931        |
|RABBIT NIGHT LIGHT                |27202        |
|MINI PAINT SET VINTAGE            |26076        |
+----------------------------------+-------------+
only showing top 10 rows

